In [28]:
import numpy as np
import agentpy as ap

import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [27]:
class Car(ap.Agent):
    
    def setup(self):    
        self.grid= self.model.grid
        self.positon = [0, 0]
        self.color = 1
        self.side = [1, 0]
        self.speed = 1
        self.crossed = False
    
    def get_location(self):
        self.position= self.grid.positions[self]
        if self.position[1] == 0:
            self.side = [0,1]
    
    def get_movement(self):
        self.get_location()
        return (self.speed*self.side[0], self.speed*self.side[1] )
    
    def get_route(self):
        self.position = self.grid.positions[self]  
        if self.position[1]==0:
            return 'HORIZONTAL'
        return 'VERTICAL'
    
class TrafficLight(ap.Agent):
        
    def setup(self):
            self.state = 1
            self.color = 3
            self.grid = self.model.grid
            self.position = [0,0]
            self.route=''
            
    def get_positions(self):
            self.position = self.grid.positions[self]
        
    def change_state(self):
            self.get_positions()
            tGrid = self.p['Grid']
            self.state= 2
            self.color = 2
            if self.model.n_cars_1>self.model.n_cars_2:
                if self.position[0] == int((tGrid/2)+1):
                    self.state = 0
                    self.color = 4
            elif self.model.n_cars_1 < self.model.n_cars_2:
                if self.position[1] == int((tGrid/2)+1):
                    self.state = 0
                    self.color=4
            else:
                if self.position[1] == int((tGrid/2)+1):
                    self.state = 0
                    self.color=4
                    

class IntersectionModel(ap.Model):
    def setup(self):
        self.con=0
        
        tGrid= self.p['Grid']
        self.grid= ap.Grid(self,[tGrid]*2,torus=True,track_empty=True)
        
        self.n_cars_1=0
        self.n_cars_2=0.
        
        n_cars = self.p['Vehicles']
        n_streets = tGrid*2
        
        self.cars= ap.AgentList(self,n_cars,Car)

        #self.street = ap.AgentList(self, n_streets, Streets)
        self.TrafficLight = ap.AgentList(self, 2, TrafficLight)
        
        self.cars.grid = self.grid
        
        street_pos = []
        tGrid = self.p['Grid']
        for i in range(tGrid*2):
            if i <= tGrid-1:
                street_pos.append((i, int(tGrid/2)))
            else:
                street_pos.append((int(tGrid/2), i - tGrid))
            
        self.grid.add_agents(self.TrafficLight, positions=[(int((tGrid/2)-1), int((tGrid/2)+1)), (int((tGrid/2)+1), int((tGrid/2)-1))])
        stop_light = self.TrafficLight
        for semaforo in stop_light:
            if self.grid.positions[semaforo][1]==0:
                semaforo.route='VERTICAL'
            else: semaforo.route='HORIZONTAL'
      
        vehicles_positions=[]
        for i in range (1,n_cars+1):
            if i%2==0:
                vehicles_positions.append((int(tGrid/2), 0))
            else:
                vehicles_positions.append((0,int(tGrid/2)))

    def step(self):
        tGrid = self.p['Grid']
        n_cars = self.p['Vehicles']
        if self.con==0:
            vehicles_positions=[]
            positions=[(0, int(tGrid/2)), (int(tGrid/2), 0)]
            
            for i in range (n_cars):
                rand_pos = np.random.randint(0, 2)
                vehicles_positions.append(positions[rand_pos])

            new_car=ap.AgentList(self,1,Car)
            self.grid.add_agents(self.cars,vehicles_positions)
            self.con+=1

        
        n_movements=0
        move_cars=0
        movimiento=True
        state=False
        moving_cars_1 = self.cars
        
        for car in moving_cars_1:
            agent_pos=self.grid.positions[car]
            for i in range(1,int((tGrid/2))):
                if (int(tGrid/2) == agent_pos[0])and(int(i) == agent_pos[1]):
                    self.n_cars_1 += 1
                    state=True
                if  (int(tGrid/2) == agent_pos[1])and(int(i) == agent_pos[0]):
                    self.n_cars_2 += 1
                    state=True
        if state:
            stop_light = self.TrafficLight
            for semaforo in stop_light:    
                if state:
                    semaforo.change_state()
                else:
                    semaforo.state=1
                    semaforo.color=3

        
       
        for agent in self.grid.agents:
            agent_pos=self.grid.positions[agent]
            movimiento=True
            if agent.type == 'Car':
                for neighbor in self.grid.neighbors(agent):
                    if agent.get_route() == 'VERTICAL':
                        if self.grid.positions[neighbor][1]==agent_pos[1]+1 and self.grid.positions[neighbor][0]==agent_pos[0]:
                            if   neighbor.type=='Car':    
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][1]==agent_pos[1] and self.grid.positions[neighbor][0]==agent_pos[0]+1:
                            if  neighbor.type=='TrafficLight':
                                if neighbor.state==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_1-=1
                                    break
                        
                    if agent.get_route() == 'VERTICAL':
                        if self.grid.positions[neighbor][0]==agent_pos[0]+1 and self.grid.positions[neighbor][1]==agent_pos[1]:
                            if  neighbor.type=='Car':
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][0]==agent_pos[0] and self.grid.positions[neighbor][1]==agent_pos[1]+1:
                            if  neighbor.type=='TrafficLight':
                                if neighbor.state==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_2-=1
                                    break
                if movimiento:
                    coordinates_move=agent.get_movement()
                    self.grid.move_by(agent,coordinates_move)
                    
    def end(seld):
        pass
    
parameters = {
    'Vehicles' : 10, 'steps' : 500, 'Grid': 25,
}

def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('color')
    #0 es verde claro, 1 es gris, 2 es rojo, 3 amarillo, 4 verde fuerte
    color_dict = {0: '#d5e5d5', 1: '#e5e5e5', 2: '#d62c2c', 3: '#FFFF00', 4: '#21d41e', None: '#7FC97F'}  # '#d5e5d5' '#d62c2c'
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    

fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))

            
                   